In [23]:
import tensorflow as tf

from azureml.core import Run
from azureml.core import Experiment, Workspace

In [24]:
one_step_reloaded = tf.saved_model.load('model/one_step_model')

states = None
next_char = tf.constant(['sejm'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

sejmie takiego słowa w sprawach ?
jeśli chodzi o reklamows ‒ dlatego , że stana nie tyle obyta .
być m


In [25]:
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='textgen')

run = Run(exp, 'textgen_1611385820_c8bf89c3')

In [26]:
model = run.register_model(model_name='textgen_model', model_path='outputs/model/')

In [27]:
model

Model(workspace=Workspace.create(name='deep-learning', subscription_id='fc814e44-1cd5-4ab5-944b-f2255f816d34', resource_group='text-generation'), name=textgen_model, id=textgen_model:5, version=5, tags={}, properties={})

In [28]:
%%writefile gen.py
import json
import numpy as np
import os
import tensorflow as tf

from azureml.core.model import Model

def init():
    global model
    model_root = os.getenv('AZUREML_MODEL_DIR')
    model_folder = 'model/one_step_model'
    model = tf.saved_model.load(os.path.join(model_root, model_folder))
 
    
def run(input_):
    json_ = json.loads(input_)
    next_char = tf.constant([json_["data"]])
    n = json_['num']

    json_1 = { "data" : [] }
    for i in range(n):
        states = None
        next_char = tf.constant([json_["data"]])
        result = [ next_char ]
        for n in range(100):
            next_char, states = model.generate_one_step(next_char, states=states)
            result.append(next_char)
        json_1["data"].append(tf.strings.join(result)[0].numpy().decode("utf-8").replace('\r\n', " "))

    return json_1



Overwriting gen.py


In [21]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_pip_package('tensorflow==2.4.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.4.1
  - azureml-defaults
- numpy
channels:
- anaconda
- conda-forge



In [30]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="gen.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'textgen', 'framework': 'TfKeras'},
                                               description='text-generation')

service = Model.deploy(workspace=ws, 
                           name='text-generation8h', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........

In [86]:
print(service.get_logs())

2021-01-23T13:26:17,617167768+00:00 - rsyslog/run 
2021-01-23T13:26:17,617167668+00:00 - iot-server/run 
2021-01-23T13:26:17,617771094+00:00 - gunicorn/run 
2021-01-23T13:26:17,635322733+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
Starting